In [1]:
# ========================
# 0. IMPORTS
# ========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, LSTM, RepeatVector, TimeDistributed, 
                                     MultiHeadAttention, LayerNormalization, Add, 
                                     Conv1D, GlobalAveragePooling1D)

from tensorflow.keras.callbacks import EarlyStopping
from einops import rearrange

import os

In [2]:
# ========================
# 1. CONFIGURATION
# ========================
INPUT_STEPS = 20
FORECAST_STEPS = 20
# PATCH_LEN = 3   # must divide INPUT_STEPS
TEST_RATIO = 0.3

# Tuning parameters
EPOCHS_LIST = [10, 20]
BATCH_SIZES = [64, 128]

# Simulation parameters
WINDOW_SIZE_SIMULATION = 12  # 6 hours → 6×6=36 steps (if 10-min data)
THRESHOLD_PERCENTILE = 95

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [24]:
# ========================
# 2. DEVICE SETUP
# ========================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU is available and will be used.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")

✅ GPU is available and will be used.


In [25]:
# ========================
# 3. LOAD AND PREPROCESS DATA
# ========================
file_path = '../rfcc_longest_active_window.csv'
df = pd.read_csv(file_path)
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df.set_index('DateTime', inplace=True)

# Clean and normalize
df.dropna(axis=1, thresh=int(0.7 * len(df)), inplace=True)
df.ffill(inplace=True)
df.bfill(inplace=True)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df.values)
df_scaled = pd.DataFrame(scaled, index=df.index, columns=df.columns).astype(np.float32)

print(f"✅ Scaled dataset shape: {df_scaled.shape}")

✅ Scaled dataset shape: (62174, 26)


In [26]:
# ========================
# 4. SEQUENTIAL TRAIN/TEST SPLIT
# ========================
split_idx = int((1 - TEST_RATIO) * len(df_scaled))
train_data = df_scaled.iloc[:split_idx]
test_data = df_scaled.iloc[split_idx:]

print(f"✅ Training samples: {len(train_data)}, Testing samples: {len(test_data)}")

# ========================
# 5. PATCH SEQUENCE GENERATOR
# ========================
def create_patch_sequences(data, input_steps, forecast_steps):
    X, y = [], []
    for i in range(len(data) - input_steps - forecast_steps):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+forecast_steps])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X_train, y_train = create_patch_sequences(train_data.values, INPUT_STEPS, FORECAST_STEPS)
X_test, y_test = create_patch_sequences(test_data.values, INPUT_STEPS, FORECAST_STEPS)

print(f"✅ Training sequences: {X_train.shape}, Testing sequences: {X_test.shape}")



# # ========================
# # 4. COMBINE TRAIN + TEST SETS
# # ========================
# # Au lieu de splitter, on prend toutes les données comme train_set
# full_train_data = df_scaled  # On utilise tout le dataset

# # On crée quand même un petit validation_set (10%) séquentiel
# val_ratio = 0.1
# split_idx = int((1 - val_ratio) * len(full_train_data))
# train_data = full_train_data.iloc[:split_idx]
# val_data = full_train_data.iloc[split_idx:]

# print(f"✅ Full training samples: {len(train_data)}, Validation samples: {len(val_data)}")

# # ========================
# # 5. PATCH SEQUENCE GENERATOR (identique)
# # ========================
# def create_patch_sequences(data, input_steps, forecast_steps):
#     X, y = [], []
#     for i in range(len(data) - input_steps - forecast_steps):
#         X.append(data[i:i+input_steps])
#         y.append(data[i+input_steps:i+input_steps+forecast_steps])
#     return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
# X_train, y_train = create_patch_sequences(train_data.values, INPUT_STEPS, FORECAST_STEPS)
# X_val, y_val = create_patch_sequences(val_data.values, INPUT_STEPS, FORECAST_STEPS)  # Remplace X_test/y_test

# print(f"✅ Training sequences: {X_train.shape}, Validation sequences: {X_val.shape}")

✅ Training samples: 43521, Testing samples: 18653
✅ Training sequences: (43481, 20, 26), Testing sequences: (18613, 20, 26)


In [27]:
# ========================
# 6. BUILD PATCHTST MODEL (CORRECTED)
# ========================
def build_patchtst(input_steps, num_features, patch_len=4, embed_dim=128, num_heads=4, num_layers=1):
    assert input_steps % patch_len == 0, f"Input steps ({input_steps}) must be divisible by patch length ({patch_len})"
    num_patches = input_steps // patch_len

    inp = Input(shape=(input_steps, num_features))
    
    # Channel Independence
    x = tf.reshape(inp, (-1, input_steps, 1))  # (batch*num_features, timesteps, 1)
    
    # Patching
    x = tf.reshape(x, (-1, num_patches, patch_len))  # (batch*num_features, num_patches, patch_len)
    x = Dense(embed_dim)(x)  # (batch*num_features, num_patches, embed_dim)

    # Transformer Blocks
    for _ in range(num_layers):
        # Self-Attention
        attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
        x = LayerNormalization()(Add()([x, attn]))
        
        # Feed Forward Network
        ffn = Dense(embed_dim * 4, activation='gelu')(x)
        ffn = Dense(embed_dim)(ffn)
        x = LayerNormalization()(Add()([x, ffn]))

    # Output Head
    x = x[:, -1, :]  # Last patch only (batch*num_features, embed_dim)
    x = tf.reshape(x, (-1, num_features, embed_dim))  # (batch, num_features, embed_dim)
    
    # Ensure output matches FORECAST_STEPS
    x = Dense(FORECAST_STEPS)(x)  # (batch, num_features, forecast_steps)
    out = tf.transpose(x, [0, 2, 1])  # (batch, forecast_steps, num_features)

    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

In [28]:
# ========================
# 7. TRAINING + TUNING (CORRECTED)
# ========================
# First verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"Input steps: {INPUT_STEPS}, Forecast steps: {FORECAST_STEPS}, Features: {X_train.shape[2]}")

# Test model output shape
test_model = build_patchtst(INPUT_STEPS, X_train.shape[2])
test_pred = test_model.predict(X_train[:1])
print(f"Test prediction shape: {test_pred.shape} (should match {y_train[:1].shape})")

assert test_pred.shape == y_train[:1].shape, \
    f"Shape mismatch! Model outputs {test_pred.shape} but y_train has {y_train[:1].shape}"

best_val_mse = np.inf
best_model = None
history_records = []

for epochs in EPOCHS_LIST:
    for batch_size in BATCH_SIZES:
        print(f"\n🔵 Training with epochs={epochs}, batch_size={batch_size}")
        
        model = build_patchtst(INPUT_STEPS, X_train.shape[2])
        es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        history = model.fit(
            X_train, y_train,
            validation_split=0.1,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[es],
            verbose=1,
            shuffle=False
        )
        
        val_preds = model.predict(X_test, batch_size=batch_size)
        
        # Flatten all except batch dimension for metrics
        val_mse = mean_squared_error(
            y_test.reshape(-1, FORECAST_STEPS * X_test.shape[2]), 
            val_preds.reshape(-1, FORECAST_STEPS * X_test.shape[2])
        )
        val_mae = mean_absolute_error(
            y_test.reshape(-1, FORECAST_STEPS * X_test.shape[2]),
            val_preds.reshape(-1, FORECAST_STEPS * X_test.shape[2])
        )

        print(f"✅ Val MSE: {val_mse:.5f}, MAE: {val_mae:.5f}")

        history_records.append({
            "epochs": epochs,
            "batch_size": batch_size,
            "val_mse": val_mse,
            "val_mae": val_mae,
            "best_epoch": np.argmin(history.history['val_loss']) + 1
        })

        if val_mse < best_val_mse:
            best_val_mse = val_mse
            best_model = model
            print("🏆 New best model!")

# Save results
history_df = pd.DataFrame(history_records)
history_df.to_csv("patchtst_tuning_history.csv", index=False)
print("\n📋 Tuning Results:")
print(history_df.sort_values('val_mse'))

if best_model:
    best_model.save("best_patchtst_model.h5")
    print("\n✅ Best model saved with:")
    print(f"- Val MSE: {best_val_mse:.5f}")
    print(f"- Best config: {history_df.loc[history_df['val_mse'].idxmin()]}")


# # ========================
# # 7. TRAINING + TUNING (CORRECTED)
# # ========================
# # First verify shapes
# print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
# print(f"Input steps: {INPUT_STEPS}, Forecast steps: {FORECAST_STEPS}, Features: {X_train.shape[2]}")

# # Test model output shape
# test_model = build_patchtst(INPUT_STEPS, X_train.shape[2])
# test_pred = test_model.predict(X_train[:1])
# print(f"Test prediction shape: {test_pred.shape} (should match {y_train[:1].shape})")

# assert test_pred.shape == y_train[:1].shape, \
#     f"Shape mismatch! Model outputs {test_pred.shape} but y_train has {y_train[:1].shape}"

# best_val_mse = np.inf
# best_model = None
# history_records = []

# for epochs in EPOCHS_LIST:
#     for batch_size in BATCH_SIZES:
#         print(f"\n🔵 Training with epochs={epochs}, batch_size={batch_size}")
        
#         model = build_patchtst(INPUT_STEPS, X_train.shape[2])
#         es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
#         history = model.fit(
#             X_train, y_train,
#             validation_split=0.1,
#             epochs=epochs,
#             batch_size=batch_size,
#             callbacks=[es],
#             verbose=1,
#             shuffle=False
#         )
        
#         val_preds = model.predict(X_val, batch_size=batch_size)
        
#         # Flatten all except batch dimension for metrics
#         val_mse = mean_squared_error(
#             y_val.reshape(-1, FORECAST_STEPS * X_val.shape[2]),  # <-- y_val au lieu de y_test
#             val_preds.reshape(-1, FORECAST_STEPS * X_val.shape[2])
#         )
#         val_mae = mean_absolute_error(
#             y_test.reshape(-1, FORECAST_STEPS * X_test.shape[2]),
#             val_preds.reshape(-1, FORECAST_STEPS * X_test.shape[2])
#         )

#         print(f"✅ Val MSE: {val_mse:.5f}, MAE: {val_mae:.5f}")

#         history_records.append({
#             "epochs": epochs,
#             "batch_size": batch_size,
#             "val_mse": val_mse,
#             "val_mae": val_mae,
#             "best_epoch": np.argmin(history.history['val_loss']) + 1
#         })

#         if val_mse < best_val_mse:
#             best_val_mse = val_mse
#             best_model = model
#             print("🏆 New best model!")

# # Save results
# history_df = pd.DataFrame(history_records)
# history_df.to_csv("patchtst_tuning_history.csv", index=False)
# print("\n📋 Tuning Results:")
# print(history_df.sort_values('val_mse'))

# if best_model:
#     best_model.save("best_patchtst_model.h5")
#     print("\n✅ Best model saved with:")
#     print(f"- Val MSE: {best_val_mse:.5f}")
#     print(f"- Best config: {history_df.loc[history_df['val_mse'].idxmin()]}")

X_train shape: (43481, 20, 26), y_train shape: (43481, 20, 26)
Input steps: 20, Forecast steps: 20, Features: 26
1/1 [==============================] - 0s 418ms/step
Test prediction shape: (1, 20, 26) (should match (1, 20, 26))

🔵 Training with epochs=10, batch_size=64
Epoch 1/10
612/612 [==============================] - 15s 22ms/step - loss: 0.0560 - val_loss: 0.0411
Epoch 2/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0430 - val_loss: 0.0424
Epoch 3/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0421 - val_loss: 0.0365
Epoch 4/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0415 - val_loss: 0.0367
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0410 - val_loss: 0.0367
Epoch 6/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0410 - val_loss: 0.0356
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0410 - val_loss: 0.0349
Epoch 8/10
612/6